In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

Using TensorFlow backend.
/home/sdv/m2bi/ajaquaniello/.conda/envs/deep_learning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sdv/m2bi/ajaquaniello/.conda/envs/deep_learning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sdv/m2bi/ajaquaniello/.conda/envs/deep_learning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 =

Selon des descriptions de radiographies, le modele va prédire si le cancer du sein est bégnin ou main.

### 1) Préparation des données

C'est une étape déterminante dans l'exactitude du réseau de neurone.

In [70]:
names = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size',
         'Uniformity of Cell Shape','Marginal Adhesion',
         'Single Epithelial Cell Size','Bare Nuclei',
         'Bland Chromatin', 'Normal Nucleoli', 'Mitoses', 'class']
dataset = pd.read_csv("../data/breast-cancer-wisconsin.data",names=names)
print(dataset)

     Sample code number  Clump Thickness  Uniformity of Cell Size  \
0               1000025                5                        1   
1               1002945                5                        4   
2               1015425                3                        1   
3               1016277                6                        8   
4               1017023                4                        1   
..                  ...              ...                      ...   
678              776715                3                        1   
679              841769                2                        1   
680              888820                5                       10   
681              897471                4                        8   
682              897471                4                        8   

     Uniformity of Cell Shape  Marginal Adhesion  Single Epithelial Cell Size  \
0                           1                  1                            2   
1        

In [71]:
#Pour que scykitlearn ne plante pas à cause de données non numériques
output_check = dataset.applymap(np.isreal).all(0)
for i in output_check:
    if (i == False):
        print("Error")
        print(output_check)
        quit()

In [72]:
dataset["class"] = dataset["class"].astype("category")
dataset["class-value"] = dataset["class"].cat.codes
#conversion en float
dataset["class-value"].astype(float)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
678    0.0
679    0.0
680    1.0
681    1.0
682    1.0
Name: class-value, Length: 683, dtype: float64

In [35]:
#vérifications
print(dataset.shape)
print(dataset.head(20))
print(dataset.describe())
print(dataset.groupby("class").size())

(683, 12)
    Sample code number  Clump Thickness  Uniformity of Cell Size  \
0              1000025                5                        1   
1              1002945                5                        4   
2              1015425                3                        1   
3              1016277                6                        8   
4              1017023                4                        1   
5              1017122                8                       10   
6              1018099                1                        1   
7              1018561                2                        1   
8              1033078                2                        1   
9              1033078                4                        2   
10             1035283                1                        1   
11             1036172                2                        1   
12             1041801                5                        3   
13             1043999                

In [75]:
X = dataset[['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size','Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mitoses']]
X = X.to_numpy()  #transform en numpy pour keras

In [76]:
Y = dataset[['class-value']]  #Recupere colonne correspondante
Y = Y.values #Recupere les valeurs
Y = Y.astype(float) #

In [21]:
print("Data values")
print(X)
print(Y)

Data values
[[ 5]
 [ 5]
 [ 3]
 [ 6]
 [ 4]
 [ 8]
 [ 1]
 [ 2]
 [ 2]
 [ 4]
 [ 1]
 [ 2]
 [ 5]
 [ 1]
 [ 8]
 [ 7]
 [ 4]
 [ 4]
 [10]
 [ 6]
 [ 7]
 [10]
 [ 3]
 [ 1]
 [ 5]
 [ 3]
 [ 5]
 [ 2]
 [ 1]
 [ 3]
 [ 2]
 [10]
 [ 2]
 [ 3]
 [ 2]
 [10]
 [ 6]
 [ 5]
 [ 2]
 [10]
 [ 6]
 [ 5]
 [10]
 [ 1]
 [ 3]
 [ 1]
 [ 4]
 [ 7]
 [ 9]
 [ 5]
 [10]
 [ 5]
 [10]
 [10]
 [ 8]
 [ 8]
 [ 5]
 [ 9]
 [ 5]
 [ 1]
 [ 9]
 [ 6]
 [ 1]
 [10]
 [ 4]
 [ 5]
 [ 8]
 [ 1]
 [ 5]
 [ 6]
 [ 1]
 [ 9]
 [10]
 [ 1]
 [ 1]
 [ 5]
 [ 3]
 [ 2]
 [ 2]
 [ 4]
 [ 5]
 [ 3]
 [ 3]
 [ 5]
 [ 3]
 [ 3]
 [ 4]
 [ 2]
 [ 1]
 [ 3]
 [ 4]
 [ 1]
 [ 2]
 [ 1]
 [ 2]
 [ 5]
 [ 9]
 [ 7]
 [10]
 [ 2]
 [ 4]
 [ 8]
 [10]
 [ 7]
 [10]
 [ 1]
 [ 1]
 [ 6]
 [ 1]
 [ 8]
 [10]
 [10]
 [ 3]
 [ 1]
 [ 8]
 [ 4]
 [ 1]
 [ 3]
 [ 1]
 [ 4]
 [10]
 [ 5]
 [ 5]
 [ 1]
 [ 7]
 [ 3]
 [ 8]
 [ 1]
 [ 5]
 [ 2]
 [ 5]
 [ 3]
 [ 3]
 [ 5]
 [ 4]
 [ 3]
 [ 4]
 [ 3]
 [ 2]
 [ 9]
 [ 1]
 [ 2]
 [ 3]
 [ 1]
 [ 3]
 [ 8]
 [ 1]
 [ 7]
 [10]
 [ 4]
 [ 1]
 [ 5]
 [ 1]
 [ 2]
 [ 9]
 [10]
 [ 4]
 [ 3]
 [ 1]
 [ 4]
 [ 5]
 [10]
 [ 3]
 [ 1]
 [ 3

In [77]:
classes = LabelEncoder()
classes.fit(Y)
classes_Y = classes.transform(Y)

In [26]:
#onehot_Y = np_utils.to_categorical(classes_Y)

print(X)
print(Y)

print(X.shape)
print(Y.shape)

[[ 5]
 [ 5]
 [ 3]
 [ 6]
 [ 4]
 [ 8]
 [ 1]
 [ 2]
 [ 2]
 [ 4]
 [ 1]
 [ 2]
 [ 5]
 [ 1]
 [ 8]
 [ 7]
 [ 4]
 [ 4]
 [10]
 [ 6]
 [ 7]
 [10]
 [ 3]
 [ 1]
 [ 5]
 [ 3]
 [ 5]
 [ 2]
 [ 1]
 [ 3]
 [ 2]
 [10]
 [ 2]
 [ 3]
 [ 2]
 [10]
 [ 6]
 [ 5]
 [ 2]
 [10]
 [ 6]
 [ 5]
 [10]
 [ 1]
 [ 3]
 [ 1]
 [ 4]
 [ 7]
 [ 9]
 [ 5]
 [10]
 [ 5]
 [10]
 [10]
 [ 8]
 [ 8]
 [ 5]
 [ 9]
 [ 5]
 [ 1]
 [ 9]
 [ 6]
 [ 1]
 [10]
 [ 4]
 [ 5]
 [ 8]
 [ 1]
 [ 5]
 [ 6]
 [ 1]
 [ 9]
 [10]
 [ 1]
 [ 1]
 [ 5]
 [ 3]
 [ 2]
 [ 2]
 [ 4]
 [ 5]
 [ 3]
 [ 3]
 [ 5]
 [ 3]
 [ 3]
 [ 4]
 [ 2]
 [ 1]
 [ 3]
 [ 4]
 [ 1]
 [ 2]
 [ 1]
 [ 2]
 [ 5]
 [ 9]
 [ 7]
 [10]
 [ 2]
 [ 4]
 [ 8]
 [10]
 [ 7]
 [10]
 [ 1]
 [ 1]
 [ 6]
 [ 1]
 [ 8]
 [10]
 [10]
 [ 3]
 [ 1]
 [ 8]
 [ 4]
 [ 1]
 [ 3]
 [ 1]
 [ 4]
 [10]
 [ 5]
 [ 5]
 [ 1]
 [ 7]
 [ 3]
 [ 8]
 [ 1]
 [ 5]
 [ 2]
 [ 5]
 [ 3]
 [ 3]
 [ 5]
 [ 4]
 [ 3]
 [ 4]
 [ 3]
 [ 2]
 [ 9]
 [ 1]
 [ 2]
 [ 3]
 [ 1]
 [ 3]
 [ 8]
 [ 1]
 [ 7]
 [10]
 [ 4]
 [ 1]
 [ 5]
 [ 1]
 [ 2]
 [ 9]
 [10]
 [ 4]
 [ 3]
 [ 1]
 [ 4]
 [ 5]
 [10]
 [ 3]
 [ 1]
 [ 3]
 [ 1]
 [ 1

### 2) Création d'un modèle

In [78]:
model = Sequential()
model.add(Dense(10,input_dim=9 , activation ="sigmoid")) #10 neurones dans la couche après la couche d'entrée
model.add(Dense(4,activation ="sigmoid")) #couche cachée 
model.add(Dense(1, activation="sigmoid")) #dernière couche

In [79]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 10)                100       
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 44        
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 5         
Total params: 149
Trainable params: 149
Non-trainable params: 0
_________________________________________________________________


### 3) Compilation

In [80]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [85]:
model.fit(X,Y,epochs=25, batch_size = 10)

Epoch 1/25
683/683 [==============================] - 0s 175us/step - loss: 0.6573 - accuracy: 0.7540
Epoch 2/25
683/683 [==============================] - 0s 137us/step - loss: 0.6216 - accuracy: 0.6530
Epoch 3/25
683/683 [==============================] - 0s 138us/step - loss: 0.5930 - accuracy: 0.6501
Epoch 4/25
683/683 [==============================] - 0s 129us/step - loss: 0.5644 - accuracy: 0.6501
Epoch 5/25
683/683 [==============================] - 0s 135us/step - loss: 0.5345 - accuracy: 0.6618
Epoch 6/25
683/683 [==============================] - 0s 140us/step - loss: 0.5029 - accuracy: 0.7291
Epoch 7/25
683/683 [==============================] - 0s 130us/step - loss: 0.4681 - accuracy: 0.8228
Epoch 8/25
683/683 [==============================] - 0s 155us/step - loss: 0.4318 - accuracy: 0.8829
Epoch 9/25
683/683 [==============================] - 0s 161us/step - loss: 0.3966 - accuracy: 0.9151
Epoch 10/25
683/683 [==============================] - 0s 129us/step - loss: 0.364

### 4) Evaluation du modele

In [86]:
evaluation = model.evaluate(X,Y) 
print("accuracy",(evaluation[1]*100))
print("loss",(evaluation[0]))

683/683 [==============================] - 0s 13us/step
accuracy 96.48609161376953
loss 0.15254346861628104


In [87]:
predictions = model.predict(X)
print("raw prediction")
for i in range(10):
	print(predictions[i])

raw prediction
[0.03915527]
[0.8558012]
[0.04551044]
[0.8764819]
[0.04160658]
[0.8783092]
[0.7468401]
[0.04250407]
[0.0405257]
[0.05606714]


In [88]:
rounded = [round(x[0]) for x in predictions]
print('Rounded row predictions')
for i in range(10):
    print(rounded[i])

Rounded row predictions
0.0
1.0
0.0
1.0
0.0
1.0
1.0
0.0
0.0
0.0


In [89]:
for i in range(10):
    distance = Y[i]-predictions[i]
    print("%s => %d (expected %d)[D=%f]"% (X[i].tolist(), predictions[i], Y[i], distance  ))

[5, 1, 1, 1, 2, 1, 3, 1, 1] => 0 (expected 0)[D=-0.039155]
[5, 4, 4, 5, 7, 10, 3, 2, 1] => 0 (expected 0)[D=-0.855801]
[3, 1, 1, 1, 2, 2, 3, 1, 1] => 0 (expected 0)[D=-0.045510]
[6, 8, 8, 1, 3, 4, 3, 7, 1] => 0 (expected 0)[D=-0.876482]
[4, 1, 1, 3, 2, 1, 3, 1, 1] => 0 (expected 0)[D=-0.041607]
[8, 10, 10, 8, 7, 10, 9, 7, 1] => 0 (expected 1)[D=0.121691]
[1, 1, 1, 1, 2, 10, 3, 1, 1] => 0 (expected 0)[D=-0.746840]
[2, 1, 2, 1, 2, 1, 3, 1, 1] => 0 (expected 0)[D=-0.042504]
[2, 1, 1, 1, 2, 1, 1, 1, 5] => 0 (expected 0)[D=-0.040526]
[4, 2, 1, 1, 2, 1, 2, 1, 1] => 0 (expected 0)[D=-0.056067]


In [96]:
from keras.wrappers.scikit_learn import KerasClassifier

### 5) Cross validation

In [102]:
def my_model():
    model = Sequential()
    model.add(Dense(10,input_dim=9 , activation ="sigmoid")) #10 neurones dans la couche après la couche d'entrée
    model.add(Dense(4,activation ="sigmoid")) #couche cachée 
    model.add(Dense(1, activation="sigmoid")) #dernière couche
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
    return model

In [104]:
training = KerasClassifier(build_fn=my_model,epochs=200,batch_size=10,verbose=0)
kfold = KFold(n_splits=2, shuffle=True)
cv_result = cross_val_score(training,X,Y,cv=kfold)
print(cv_result)
print("%.2f%%(%2d%%)"%(cv_result.mean()*100, cv_result.std()*100))

[0.95029241 0.97067451]
96.05%( 1%)


### 6) Multiclasse

Le one hot encoding est important pour le multiclasse.
On va changer la fonction de loss (en categorical cross_entropy)

In [111]:
def other_model():
    model = Sequential()
    model.add(Dense(10,input_dim=9 , activation ="sigmoid")) #10 neurones dans la couche après la couche d'entrée
    model.add(Dense(4,activation ="sigmoid")) #couche cachée 
    model.add(Dense(2, activation="sigmoid")) #dernière couche
    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
    return model

In [112]:
classes = LabelEncoder()
classes.fit(Y)
classes_Y = classes.transform(Y)
onehot_Y = np_utils.to_categorical(classes_Y)

/home/sdv/m2bi/ajaquaniello/.conda/envs/deep_learning/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sdv/m2bi/ajaquaniello/.conda/envs/deep_learning/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [113]:
other_model = other_model()

In [115]:
other_model.fit(X,onehot_Y,epochs=25, batch_size = 10)

Epoch 1/25
683/683 [==============================] - 0s 685us/step - loss: 0.6993 - accuracy: 0.4495
Epoch 2/25
683/683 [==============================] - 0s 176us/step - loss: 0.6468 - accuracy: 0.8155
Epoch 3/25
683/683 [==============================] - 0s 157us/step - loss: 0.6114 - accuracy: 0.7101
Epoch 4/25
683/683 [==============================] - 0s 151us/step - loss: 0.5825 - accuracy: 0.6706
Epoch 5/25
683/683 [==============================] - 0s 151us/step - loss: 0.5544 - accuracy: 0.6881
Epoch 6/25
683/683 [==============================] - 0s 164us/step - loss: 0.5244 - accuracy: 0.7335
Epoch 7/25
683/683 [==============================] - 0s 168us/step - loss: 0.4912 - accuracy: 0.8287
Epoch 8/25
683/683 [==============================] - 0s 166us/step - loss: 0.4542 - accuracy: 0.8799
Epoch 9/25
683/683 [==============================] - 0s 160us/step - loss: 0.4150 - accuracy: 0.9107
Epoch 10/25
683/683 [==============================] - 0s 173us/step - loss: 0.376

In [119]:
training = KerasClassifier(build_fn=other_model(),epochs=200,batch_size=10,verbose=0)
kfold = KFold(n_splits=2, shuffle=True)
cv_result = cross_val_score(training,X,onehot_Y,cv=kfold)
print(cv_result)
print("%.2f%%(%2d%%)"%(cv_result.mean()*100, cv_result.std()*100))

TypeError: __call__() missing 1 required positional argument: 'inputs'

### 7) Nouveau jeu de données: les iris

4 descripteurs donc 4 neurones dans la couche d'entrée.
3 classes donc 3 neurones dans la couche de sortie;

In [137]:
names = [1,2,3,4,"race"]
iris = pd.read_csv("../data/iris.data", names = names)

In [138]:
iris["race"] = iris["race"].astype("category")
iris["race-value"] = iris["race"].cat.codes
#conversion en float
iris["race-value"].astype(float)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
145    2.0
146    2.0
147    2.0
148    2.0
149    2.0
Name: race-value, Length: 150, dtype: float64

In [133]:
X = iris[[1,2,3,4]]
X = X.to_numpy()

In [144]:
Y = iris[['race-value']]  #Recupere colonne correspondante
Y = Y.values #Recupere les valeurs
Y = Y.astype(float) #

In [148]:
classes = LabelEncoder()
classes.fit(Y)
classes_Y = classes.transform(Y)

/home/sdv/m2bi/ajaquaniello/.conda/envs/deep_learning/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sdv/m2bi/ajaquaniello/.conda/envs/deep_learning/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [150]:
one_hot_Y = np_utils.to_categorical(classes_Y)

In [167]:
def iris_model():
    model = Sequential()
    model.add(Dense(10,input_dim = 4 , activation ="sigmoid")) #10 neurones dans la couche après la couche d'entrée
    model.add(Dense(5,activation ="sigmoid")) #couche cachée 
    model.add(Dense(3, activation="sigmoid")) #dernière couche
    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
    return model

In [168]:
iris_model = iris_model()

In [169]:
iris_model.fit(X,one_hot_Y,epochs=100, batch_size = 10)

Epoch 1/100
150/150 [==============================] - 0s 3ms/step - loss: 1.1871 - accuracy: 0.3333
Epoch 2/100
150/150 [==============================] - 0s 265us/step - loss: 1.1727 - accuracy: 0.3333
Epoch 3/100
150/150 [==============================] - 0s 313us/step - loss: 1.1603 - accuracy: 0.3333
Epoch 4/100
150/150 [==============================] - 0s 327us/step - loss: 1.1504 - accuracy: 0.3333
Epoch 5/100
150/150 [==============================] - 0s 211us/step - loss: 1.1410 - accuracy: 0.3333
Epoch 6/100
150/150 [==============================] - 0s 294us/step - loss: 1.1337 - accuracy: 0.3333
Epoch 7/100
150/150 [==============================] - 0s 248us/step - loss: 1.1276 - accuracy: 0.3333
Epoch 8/100
150/150 [==============================] - 0s 224us/step - loss: 1.1221 - accuracy: 0.3333
Epoch 9/100
150/150 [==============================] - 0s 190us/step - loss: 1.1167 - accuracy: 0.3400
Epoch 10/100
150/150 [==============================] - 0s 197us/step - los

In [173]:
def other_model():
    model = Sequential()
    model.add(Dense(10,input_dim = 4 , activation ="sigmoid")) #10 neurones dans la couche après la couche d'entrée
    model.add(Dense(5,activation ="sigmoid")) #couche cachée 
    model.add(Dense(3, activation="sigmoid")) #dernière couche
    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
    return model

In [176]:
training = KerasClassifier(build_fn = other_model,epochs=200,batch_size=10,verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
cv_result = cross_val_score(training,X,one_hot_Y,cv=kfold)
print("Cross val: {}".format(cv_result))
print("%.2f%%(%2d%%)"%(cv_result.mean()*100, cv_result.std()*100))

Cross val: [1.         1.         0.93333334 1.         1.         1.
 1.         0.93333334 1.         0.93333334]
98.00%( 3%)
